# 读入数据

In [1]:
import pandas as pd
data=pd.read_csv('../data/train1.csv')
data1=pd.read_csv('../data/train2.csv')
print(data.head())
print(data1.head())

                url  label
0  www.jcxueshu.com      2
1    www.lazxsf.com      6
2     www.741bb.com      2
3       492377a.com      2
4    www.0505se.com      6
                   url  label
0        www.fx9q.com      10
1        m.kameng.com      10
2  www.zhudaitong.net      10
3     0571jiekuan.com      10
4   www.gancuidai.com      10


In [2]:
merged_df = pd.concat([data, data1], axis=0)
# merged_df =data 
# print(len(data),len(data1),len(merged_df))
# print(data.shape,data1.shape,merged_df.shape)
print(type(merged_df['url'][0]))

<class 'pandas.core.series.Series'>


In [3]:
print(merged_df['url'][0])

0    www.jcxueshu.com
0       www.fx9q.com 
Name: url, dtype: object


# 数据预处理

In [4]:
# 根据label的值将数据拆分成不同的组
grouped = merged_df.groupby('label')

data=[]
# 遍历每个分组，将分组数据保存在一个新的DataFrame中
for label, group in grouped:
    # 构造新的DataFrame
    data.append(pd.DataFrame({'url': group['url'], 'label': group['label']}))


In [6]:
# 按类别整理数据集
for i in range(len(data)):
    print(f'label : {i} len: {len(data[i])}'  )

label : 0 len: 1033666
label : 1 len: 22
label : 2 len: 17779
label : 3 len: 173
label : 4 len: 4658
label : 5 len: 2
label : 6 len: 2603
label : 7 len: 11
label : 8 len: 883
label : 9 len: 595
label : 10 len: 998
label : 11 len: 549
label : 12 len: 7


In [8]:
import numpy as np
from tqdm import *
# 定义阈值
threshold = 1000

data_pd = []

# 遍历每个分组
for i in trange(len(data)):
    if(len(data[i]) > threshold ):
        data_pd.append(data[i].sample(n=threshold, replace=True)) 
        
        print(f'label : {i} len: {len(data[i])}')
for i in trange(len(data_pd)):
    print(data_pd[i]['label'])
                

100%|██████████| 13/13 [00:00<00:00, 3059.47it/s]


label : 0 len: 1033666
label : 2 len: 17779
label : 4 len: 4658
label : 6 len: 2603


100%|██████████| 4/4 [00:00<00:00, 890.75it/s]

47257      0
189891     0
909488     0
226660     0
947831     0
          ..
1028272    0
84190      0
859040     0
397127     0
168922     0
Name: label, Length: 1000, dtype: int64
6151    2
1866    2
224     2
4268    2
2548    2
       ..
4618    2
8307    2
8166    2
2142    2
4751    2
Name: label, Length: 1000, dtype: int64
11663    4
9228     4
12799    4
11401    4
9411     4
        ..
9302     4
12117    4
12800    4
9005     4
13031    4
Name: label, Length: 1000, dtype: int64
3141     6
14309    6
10772    6
14156    6
4253     6
        ..
10245    6
12210    6
9877     6
10559    6
7953     6
Name: label, Length: 1000, dtype: int64


In [5]:
import re
def url_change(string):
    tensor = []
    string = string.replace('www.','').replace('http://','').replace('https://','')
    
    total = len(string)
    tensor.append(total)#字符的长度
    tensor.append(string.count('.'))#.的个数
    upper_num = sum(1 for c in string if c.isupper())
    lower_num = sum(1 for c in string if c.islower())
    num = sum(1 for c in string if c.isdigit())
    other = string.count('#') + string.count('-') + string.count('@')

    tensor.append(upper_num)#大写字符的个数
    tensor.append(lower_num)#小写字符的个数
    tensor.append(num)#数字的个数
    tensor.append(other)#特殊字符的个数
    tensor.append(string.count('/'))#分隔符的个数

    delimiter = "/"
    split_string = string.split(delimiter)
    max_length = max(len(i) for i in split_string) if len(split_string) > 1 else len(string)
    tensor.append(max_length)#分隔符之间的最大长度

    letters_pattern = re.compile(r'[a-zA-Z]+')
    digits_pattern = re.compile(r'\d+')

    letters_matches = letters_pattern.findall(string)
    digits_matches = digits_pattern.findall(string)

    max_letters_length = max(len(i) for i in letters_matches) if letters_matches else 0
    max_digits_length = max(len(i) for i in digits_matches) if digits_matches else 0

    tensor.append(max_letters_length)#最大连续字母长度
    tensor.append(max_digits_length)#最大连续数字长度

    tensor.append(upper_num/total)#大写字符占比
    tensor.append(lower_num/total)#小写字符占比
    tensor.append(num/total)#数字占比
    tensor.append(other/total)#特殊字符占比

    num_to_alpha_count = len(re.findall(r'\d(?=[a-zA-Z])', string))
    alpha_to_num_count = len(re.findall(r'[a-zA-Z](?=\d)', string))

    total_count = num_to_alpha_count + alpha_to_num_count
    frequency = total_count / len(string)
    tensor.append(frequency)#字符和数字的转换频率
    return tensor

In [6]:
## 此处所引入的包大部分为下文机器学习算法
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score,f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


def split_to_columns(row):
    return pd.Series(row['url'],index=[f'FY{i+1:02d}'for i in range(len(row['url']))])

def data_process(temp,threshold):
    data1 = []
    data1.append(data[temp].sample(n=threshold, replace=True))
    data1[0]['label']=0
    data1.append(pd.concat([data[i].sample(n=threshold//12, replace=True) for i in range(12) if(i!=temp)],axis=0))
    #还需要调整一下
    data1[1]['label']=1
    merged_df = pd.concat([data1[0], data1[1]], axis=0)
    merged_df['url'] = merged_df['url'].apply(lambda x: url_change(x))
    #print(len(merged_df['url']))
    merged_df = pd.concat([merged_df,merged_df.apply(split_to_columns,axis=1)],axis=1)
    merged_df = merged_df.drop('url',axis=1)
    X=merged_df[['FY01','FY02', 'FY03', 'FY04', 'FY05', 'FY06', 'FY07', 'FY08', 'FY09', 'FY10','FY11', 'FY12', 'FY13', 'FY14', 'FY15']]
    Y=merged_df['label']
    return X,Y
    #print(merged_df)
def class_model(X,Y):
    tr_x,te_x,tr_y,te_y=train_test_split(X,Y,test_size=0.2,random_state=5)
    rf=RandomForestClassifier(max_depth=30,n_estimators=1000,random_state=0)
    rf.fit(tr_x,tr_y)
    
    print("训练集准确度:{:.3f}".format(rf.score(tr_x,tr_y)))
    print("测试集准确度:{:.3f}".format(rf.score(te_x,te_y)))
   # print("F-score",f1_score(te_y,y_pred,average="weighted"))
    
    return rf

model = []
for i in trange(13):
    threshold = 1000
    if(len(data[i])>500):
        threshold = 1000
    else:
        threshold = 500
    print('model',i)
    X,Y = data_process(i,threshold)
    model.append(class_model(X,Y))
    #locals()['model_'+str(i)] = class_model(X,Y)
    

  0%|          | 0/13 [00:00<?, ?it/s]

model 0


  8%|▊         | 1/13 [00:02<00:34,  2.88s/it]

训练集准确度:0.988
测试集准确度:0.950
model 1


 15%|█▌        | 2/13 [00:05<00:27,  2.50s/it]

...

训练集准确度:0.918
测试集准确度:0.875
model 11


 92%|█████████▏| 12/13 [00:35<00:03,  3.13s/it]

训练集准确度:0.886
测试集准确度:0.851
model 12


100%|██████████| 13/13 [00:38<00:00,  2.94s/it]

训练集准确度:1.000
测试集准确度:0.985


In [10]:
import warnings
warnings.filterwarnings("ignore")
order = [10,11,0,2,4,6,8,9,3,1,5,7,12]
order = [0,2,4,6,10,8,9,11,3,1,7,12,5]
p_tho = 0.8
test_example = url_change('4399.com')
def combine_model(text_example,model,order):
    for i in range(13):
        predict = model[order[i]].predict_proba(np.array(test_example).reshape(1, -1))[0]
#         print('model'+str(order[i]),predict)
        if predict[0]>0.7:
            return order[i]
    return 0
# print(test_example)
combine_model(test_example,model,order)


8

In [8]:
#测试集
test_data = data[0][0:10000]
for i in trange(13):
    temp = data[i]
    if len(data[i])>10000:
        temp = data[i][0:10000]
#     test_data.append(temp,ignore_index=True)
    test_data = pd.concat([test_data,temp],axis=0,ignore_index=True)


# test_data['url'] = test_data['url'].apply(lambda x: url_change(x))

# test_data = pd.concat([test_data,test_data.apply(split_to_columns,axis=1)],axis=1)
# test_data = test_data.drop('url',axis=1)
# X=test_data[['FY01','FY02', 'FY03', 'FY04', 'FY05', 'FY06', 'FY07', 'FY08', 'FY09', 'FY10','FY11', 'FY12', 'FY13', 'FY14', 'FY15']]
# Y=test_data['label']

print(test_data)



100%|██████████| 13/13 [00:00<00:00, 921.53it/s]

                                                  url  label
0                             a55mm.yangfangliang.top      0
1              1656686672.1898585287.sngdia.imtmp.net      0
2          only-809761-112-17-235-21.nstool.yqkk.link      0
3      only-747549-211-138-116-182.nstool.netease.com      0
4                                             553.jpg      0
...                                               ...    ...
40496                                    www.newrc.cn     12
40497                                    bbs.mnytt.cn     12
40498                                    www.06fj.com     12
40499                                  sandy.cesy.top     12
40500                                      bzsclb1.cn     12

[40501 rows x 2 columns]


In [11]:
import random
count = 0 
ans = [0 for i in range(13)]
acc = [0 for i in range(13)]
total = [0 for i in range(13)]
sample_number = 1000
# for i in trange(0,len(test_data),len(test_data)//100):
for i in tqdm([random.randint(0,len(test_data)) for j in range(sample_number)]):
    url = test_data.loc[i,'url']
    label = test_data.loc[i,'label']
    test_example = url_change(url)
    pre = combine_model(test_example,model,order)
    print(pre,label,i)
    if(pre ==  label):
        count+=1
        acc[label]+=1
    ans[label]+=1
    total[pre]+=1
print(sum(acc)/(sample_number))

  0%|          | 3/1000 [00:00<01:42,  9.74it/s]

0 0 13893
0 0 3833
0 0 4866


  1%|          | 6/1000 [00:00<01:47,  9.21it/s]

2 2 27510
0 0 18582
0 0 2023


  1%|          | 8/1000 [00:00<01:42,  9.66it/s]

...

6 0 3706
0 0 2521


100%|█████████▉| 998/1000 [06:15<00:00,  3.38it/s]

11 0 18609
0 0 7480
0 0 1742


100%|██████████| 1000/1000 [06:15<00:00,  2.66it/s]

0 0 14951
0 0 15538
0.64


In [13]:
print(sum(acc)/sample_number)

print(acc)
print(ans)
print(total)

0.64
[456, 2, 94, 1, 49, 0, 19, 0, 6, 7, 4, 2, 0]
[492, 2, 266, 5, 123, 0, 52, 1, 15, 12, 19, 13, 0]
[477, 11, 142, 11, 89, 34, 66, 95, 36, 21, 12, 6, 0]


# 数据编码

In [7]:
import re
def url_change(string):
    tensor = []
    string = string.replace('www.','').replace('http://','').replace('https://','')
    
    total = len(string)
    tensor.append(total)#字符的长度
    tensor.append(string.count('.'))#.的个数
    upper_num = sum(1 for c in string if c.isupper())
    lower_num = sum(1 for c in string if c.islower())
    num = sum(1 for c in string if c.isdigit())
    other = string.count('#') + string.count('-') + string.count('@')

    tensor.append(upper_num)#大写字符的个数
    tensor.append(lower_num)#小写字符的个数
    tensor.append(num)#数字的个数
    tensor.append(other)#特殊字符的个数
    tensor.append(string.count('/'))#分隔符的个数

    delimiter = "/"
    split_string = string.split(delimiter)
    max_length = max(len(i) for i in split_string) if len(split_string) > 1 else len(string)
    tensor.append(max_length)#分隔符之间的最大长度

    letters_pattern = re.compile(r'[a-zA-Z]+')
    digits_pattern = re.compile(r'\d+')

    letters_matches = letters_pattern.findall(string)
    digits_matches = digits_pattern.findall(string)

    max_letters_length = max(len(i) for i in letters_matches) if letters_matches else 0
    max_digits_length = max(len(i) for i in digits_matches) if digits_matches else 0

    tensor.append(max_letters_length)#最大连续字母长度
    tensor.append(max_digits_length)#最大连续数字长度

    tensor.append(upper_num/total)#大写字符占比
    tensor.append(lower_num/total)#小写字符占比
    tensor.append(num/total)#数字占比
    tensor.append(other/total)#特殊字符占比

    num_to_alpha_count = len(re.findall(r'\d(?=[a-zA-Z])', string))
    alpha_to_num_count = len(re.findall(r'[a-zA-Z](?=\d)', string))

    total_count = num_to_alpha_count + alpha_to_num_count
    frequency = total_count / len(string)
    tensor.append(frequency)#字符和数字的转换频率
    return tensor

In [165]:
merged_df['url'] = merged_df['url'].apply(lambda x: url_change(x))
print(len(merged_df['url']))

19163


In [177]:
print(type(merged_df))

<class 'pandas.core.frame.DataFrame'>


In [178]:
X=merged_df['url']
Y=merged_df['label']
print(Y)

955468    0
967101    0
221936    0
426205    0
189256    0
         ..
3159      1
5663      1
8606      1
857       1
836       1
Name: label, Length: 19163, dtype: int64


In [182]:
merged_df.columns

Index(['label', 'FY01', 'FY02', 'FY03', 'FY04', 'FY05', 'FY06', 'FY07', 'FY08',
       'FY09', 'FY10', 'FY11', 'FY12', 'FY13', 'FY14', 'FY15'],
      dtype='object')

In [168]:
import pandas as pd
import torch
from torch.utils.data import Dataset,DataLoader, WeightedRandomSampler, TensorDataset

import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split


train,test=train_test_split(merged_df,test_size=0.2,random_state=5)

class MyDataset(Dataset):
    def __init__(self, data):
        self.inputs = data['url'].values
        self.targets = data['label'].values.astype('int')
    
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, index):
        x = torch.tensor(self.inputs[index])
        y = torch.tensor(self.targets[index])
        return x, y

train_dataset = MyDataset(train)
test_dataset = MyDataset(test)

# # 计算每个类别的数量
# label_counts = train['label'].value_counts().to_dict()
# # 计算每个类别的权重
# total = sum(label_counts.values())
# class_weights = {label: total/count for label, count in label_counts.items()}
# # 创建WeightedRandomSampler对象
# weights = [class_weights[label] for label in train['label']]
# sampler = WeightedRandomSampler(weights, len(weights))



batch_size = 128
num_workers = 4
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,shuffle=True)
val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers,shuffle=True)                           
#tr_x,te_x,tr_y,te_y=train_test_split(X,Y,test_size=0.1,random_state=42)



In [169]:
class Accumulator:
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        """Defined in :numref:`sec_utils`"""
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
numpy = lambda x, *args, **kwargs: x.detach().numpy(*args, **kwargs)
size = lambda x, *args, **kwargs: x.numel(*args, **kwargs)
reshape = lambda x, *args, **kwargs: x.reshape(*args, **kwargs)
to = lambda x, *args, **kwargs: x.to(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)
transpose = lambda x, *args, **kwargs: x.t(*args, **kwargs)
reduce_mean = lambda x, *args, **kwargs: x.mean(*args, **kwargs)
expand_dims = lambda x, *args, **kwargs: x.unsqueeze(*args, **kwargs)
swapaxes = lambda x, *args, **kwargs: x.swapaxes(*args, **kwargs)
repeat = lambda x, *args, **kwargs: x.repeat(*args, **kwargs)

def accuracy(y_hat, y):
    """Compute the number of correct predictions.
    Defined in :numref:`sec_utils`"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))


In [170]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """Compute the accuracy for a model on a dataset using a GPU.
    Defined in :numref:`sec_utils`"""
    if isinstance(net, nn.Module):
        net.eval()  # Set the model to evaluation mode
        if not device:
            device = next(iter(net.parameters())).device
    # No. of correct predictions, no. of predictions
    metric = Accumulator(2)

    with torch.no_grad():
        for X, y in data_iter:
#             X = X.unsqueeze(1)
            if isinstance(X, list):
                # Required for BERT Fine-tuning (to be covered later)
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(accuracy(net(X), y), size(y))
    return metric[0] / metric[1]

In [171]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, 1)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64*3, 3)
        self.pool2 = nn.MaxPool1d(5)
        self.fc1 = nn.Linear(64*3, 128)
        self.fc2 = nn.Linear(128, 13)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)                   # 输入x的shape为(batch_size, 1, 15)，其中1表示输入通道数为1
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)
        x = x.view(-1, 64*3)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        return x
    
    
class linear_Net(nn.Module):
    def __init__(self):
        super(linear_Net, self).__init__()
        self.fc1 = nn.Linear(15, 32)
#         self.fc2 = nn.Linear(64, 128)
#         self.fc3 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(32, 13)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x= self.fc1(x)
#         x= self.fc2(x)
#         x= self.fc3(x)
        x= self.relu(x)
        x= self.fc4(x)

        x= self.softmax(x)
        
        return x
    
X = torch.randn(2048,15)
print(linear_Net()(X).shape)


torch.Size([2048, 13])


In [175]:
from tqdm import *
import torch.nn as nn
net = nn.Sequential(

        nn.Linear(15, 32),
        nn.ReLU(),
        nn.Linear(32, 13), 
        nn.Softmax(dim=1)
        )



batch_size, lr, num_epochs = 2, 0.1, 10

def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
#     """用GPU训练模型(在第六章定义)"""
#     def init_weights(m):
#         if type(m) == nn.Linear or type(m) == nn.Conv2d:
#             nn.init.xavier_uniform_(m.weight)
#     net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    num_batches =  len(train_iter)
    for epoch in trange(num_epochs):
        # 训练损失之和，训练准确率之和，样本数
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in (enumerate(train_iter)):
#             X = X.unsqueeze(1)
#             X = torch.randn(2048,1,15)
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
           # _,pre_label = torch.max(y_hat,dim=1)
           # print(f'lable:{pre_label}')
            
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')




In [176]:
lr, num_epochs = 0.9, 50
train_ch6(net, train_loader, val_loader, num_epochs, lr,torch.device("cuda:0"))

training on cuda:0


  2%|▏         | 1/50 [00:01<01:12,  1.49s/it]

loss 2.174, train acc 0.515, test acc 0.504


  4%|▍         | 2/50 [00:02<01:05,  1.37s/it]

loss 2.163, train acc 0.526, test acc 0.504


...

loss 2.163, train acc 0.526, test acc 0.504


 98%|█████████▊| 49/50 [01:04<00:01,  1.26s/it]

loss 2.163, train acc 0.526, test acc 0.504


100%|██████████| 50/50 [01:05<00:00,  1.31s/it]

loss 2.163, train acc 0.526, test acc 0.504
loss 2.163, train acc 0.526, test acc 0.504


In [120]:
import numpy as np
test_example = url_change('www.baidu.com')

x = torch.Tensor(test_example)
x = x.unsqueeze(0)
net.to('cpu')
print(x.shape)
print(type(x))
net(x)#url为01档的概率
print(net(x))
# np.argmax(model.predict(x), axis=1)[0]#url为0档

torch.Size([1, 15])
<class 'torch.Tensor'>
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       grad_fn=<ReluBackward0>)


In [186]:
# 此处所引入的包大部分为下文机器学习算法
import pandas as pd
from numpy import *
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score,recall_score,f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split


tr_x,te_x,tr_y,te_y=train_test_split(X,Y,test_size=0.2,random_state=5)


model=MLPClassifier(hidden_layer_sizes=10,max_iter=1000).fit(tr_x,tr_y)
print("多层感知机:")
print("训练集准确度:{:.3f}".format(model.score(tr_x,tr_y)))
print("测试集准确度:{:.3f}".format(model.score(te_x,te_y)))
y_pred = model.predict(te_x)
print("平均绝对误差:",mean_absolute_error(te_y, y_pred))
# 准确率，召回率，F-score评价
print("ACC",accuracy_score(te_y,y_pred))
print("REC",recall_score(te_y,y_pred,average="weighted"))
print("F-score",f1_score(te_y,y_pred,average="weighted"))


print("\n逻辑回归:")
logreg = LogisticRegression(C=1,solver='liblinear',multi_class ='auto')
logreg.fit(tr_x, tr_y)
score = logreg.score(tr_x, tr_y)
score2 = logreg.score(te_x, te_y)
print("训练集准确度:{:.3f}".format(logreg.score(tr_x,tr_y)))
print("测试集准确度:{:.3f}".format(logreg.score(te_x,te_y)))
y_pred = logreg.predict(te_x)
print("平均绝对误差:",mean_absolute_error(te_y, y_pred))
print("ACC",accuracy_score(te_y,y_pred))
print("REC",recall_score(te_y,y_pred,average="weighted"))
print("F-score",f1_score(te_y,y_pred,average="weighted"))


print("\n决策树:")
tree=DecisionTreeClassifier(max_depth=50,random_state=0)
tree.fit(tr_x,tr_y)
y_pred = tree.predict(te_x)
print("训练集准确度:{:.3f}".format(tree.score(tr_x,tr_y)))
print("测试集准确度:{:.3f}".format(tree.score(te_x,te_y)))
print("平均绝对误差:",mean_absolute_error(te_y, y_pred))
print("ACC",accuracy_score(te_y,y_pred))
print("REC",recall_score(te_y,y_pred,average="weighted"))
print("F-score",f1_score(te_y,y_pred,average="weighted"))

print("\n随机森林:")
rf=RandomForestClassifier(max_depth=20,n_estimators=1000,random_state=0)
rf.fit(tr_x,tr_y)
print("训练集准确度:{:.3f}".format(rf.score(tr_x,tr_y)))
print("测试集准确度:{:.3f}".format(rf.score(te_x,te_y)))
y_pred = rf.predict(te_x)
print("平均绝对误差:",mean_absolute_error(te_y, y_pred))
print("ACC",accuracy_score(te_y,y_pred))
print("REC",recall_score(te_y,y_pred,average="weighted"))
print("F-score",f1_score(te_y,y_pred,average="weighted"))


print("\nLGBM:")
lgb_model=lgb.LGBMClassifier()
lgb_model.fit(tr_x,tr_y)
print("训练集准确度:{:.3f}".format(lgb_model.score(tr_x,tr_y)))
print("测试集准确度:{:.3f}".format(lgb_model.score(te_x,te_y)))
y_pred = lgb_model.predict(te_x)
print("平均绝对误差:",mean_absolute_error(te_y, y_pred))
print("ACC",accuracy_score(te_y,y_pred))
print("REC",recall_score(te_y,y_pred,average="weighted"))
print("F-score",f1_score(te_y,y_pred,average="weighted"))


print("\nXGBOOST:")
xgb_model=xgb.XGBClassifier()
xgb_model.fit(tr_x,tr_y)
print("训练集准确度:{:.3f}".format(xgb_model.score(tr_x,tr_y)))
print("测试集准确度:{:.3f}".format(xgb_model.score(te_x,te_y)))
y_pred = xgb_model.predict(te_x)
print("平均绝对误差:",mean_absolute_error(te_y, y_pred))
print("ACC",accuracy_score(te_y,y_pred))
print("REC",recall_score(te_y,y_pred,average="weighted"))
print("F-score",f1_score(te_y,y_pred,average="weighted"))


多层感知机:
训练集准确度:0.944
测试集准确度:0.944
平均绝对误差: 0.05635272632402818
ACC 0.9436472736759718
REC 0.9436472736759718
F-score 0.9435986360606695

逻辑回归:
训练集准确度:0.930
测试集准确度:0.929
平均绝对误差: 0.07122358465953561
ACC 0.9287764153404644
REC 0.9287764153404644
F-score 0.9287322703092324

决策树:
训练集准确度:0.978
测试集准确度:0.955
平均绝对误差: 0.04513435950952257
ACC 0.9548656404904774
REC 0.9548656404904774
F-score 0.954859643412037

随机森林:
训练集准确度:0.978
测试集准确度:0.959
平均绝对误差: 0.04122097573702061
ACC 0.9587790242629793
REC 0.9587790242629793
F-score 0.9587727389796422

LGBM:
训练集准确度:0.971
测试集准确度:0.958
平均绝对误差: 0.04226454474302113
ACC 0.9577354552569789
REC 0.9577354552569789
F-score 0.9577265477910683

XGBOOST:
训练集准确度:0.975
测试集准确度:0.959
平均绝对误差: 0.04122097573702061
ACC 0.9587790242629793
REC 0.9587790242629793
F-score 0.9587743048119785


In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


# 学习曲线函数

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("game num")
    plt.ylabel("score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.legend(loc="best")
    plt.show()
    return plt

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
plot_learning_curve(logreg, "LogisticRegression", tr_x, tr_y, ylim=None, cv=cv, n_jobs=1)
plot_learning_curve(tree, "DecisionTreeClassifier", tr_x, tr_y, ylim=None, cv=None, n_jobs=1)
plot_learning_curve(rf, "RandomForestClassifier", tr_x, tr_y, ylim=None, cv=None, n_jobs=1)
plot_learning_curve(model, "MLPClassifier", tr_x, tr_y, ylim=None, cv=None, n_jobs=1)

plot_learning_curve(lgb_model, "LGBM", tr_x, tr_y, ylim=None, cv=None, n_jobs=1)
plot_learning_curve(xgb_model, "XGBOOST", tr_x, tr_y, ylim=None, cv=None, n_jobs=1)


In [187]:
from keras.models import Sequential
from keras.layers import *
from keras.layers import Conv1D
from keras.optimizers import Adam, SGD, RMSprop
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau

In [188]:
y=to_categorical(Y.values)
x = np.expand_dims(X.values, axis=-1)
print(x.shape, y.shape)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(19163, 15, 1) (19163, 2)
(15330, 15, 1) (3833, 15, 1) (15330, 2) (3833, 2)


In [192]:
# 建模
model = Sequential()
model.add(Conv1D(filters=4, kernel_size=3, padding='same', input_shape=(x.shape[1], 1), activation='tanh', strides=2))
model.add(Conv1D(filters=8, kernel_size=6, padding='same', activation='tanh', strides=2))
model.add(Conv1D(filters=16, kernel_size=9, padding='same', activation='tanh', strides=2))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 8, 4)              16        
                                                                 
 conv1d_7 (Conv1D)           (None, 4, 8)              200       
                                                                 
 conv1d_8 (Conv1D)           (None, 2, 16)             1168      
                                                                 
 flatten_2 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 1,450
Trainable params: 1,450
Non-trainable params: 0
_________________________________________________________________


In [193]:

# 优化器选择Adam 学习率为0.0001 损失函数使用交叉熵 训练200次
model.compile(optimizer=Adam(0.0001), loss=categorical_crossentropy, metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, verbose=1, batch_size=64, callbacks=[
    ModelCheckpoint('mode5l.h5', save_best_only=True, verbose=1, monitor='val_acc'),
    ReduceLROnPlateau(factor=0.9, patience=5, verbose=2)
])

Epoch 1/100
240/240 [==============================] - 2s 5ms/step - loss: 0.5775 - accuracy: 0.7486 - val_loss: 0.5217 - val_accuracy: 0.7636 - lr: 1.0000e-04
Epoch 2/100
240/240 [==============================] - 1s 5ms/step - loss: 0.4767 - accuracy: 0.7863 - val_loss: 0.4627 - val_accuracy: 0.7668 - lr: 1.0000e-04
Epoch 3/100
240/240 [==============================] - 1s 4ms/step - loss: 0.4271 - accuracy: 0.7918 - val_loss: 0.4249 - val_accuracy: 0.7910 - lr: 1.0000e-04
Epoch 4/100
240/240 [==============================] - 1s 5ms/step - loss: 0.3894 - accuracy: 0.8210 - val_loss: 0.3923 - val_accuracy: 0.8437 - lr: 1.0000e-04
Epoch 5/100
240/240 [==============================] - 1s 4ms/step - loss: 0.3560 - accuracy: 0.8594 - val_loss: 0.3629 - val_accuracy: 0.8646 - lr: 1.0000e-04
Epoch 6/100
240/240 [==============================] - 1s 4ms/step - loss: 0.3273 - accuracy: 0.8823 - val_loss: 0.3368 - val_accuracy: 0.8729 - lr: 1.0000e-04
Epoch 7/100
240/240 [===================

In [207]:
test_example = url_change('https://asd123asda123.zhihu.com')
x = np.expand_dims(np.array([test_example,test_example]), axis=-1)
model.predict(x)[0]#url为01档的概率


k +=0.005
0.9-0.95

1/1 [==============================] - 0s 22ms/step


array([0.5747215 , 0.42527848], dtype=float32)

In [197]:
np.argmax(model.predict(x), axis=1)[0]#url为0档

1/1 [==============================] - 0s 24ms/step


1

In [ ]:
from sklearn.metrics import classification_report,f1_score
pre_test = model.predict(x_test)

In [ ]:
pre_test = model.predict(x_test)
print(f1_score(np.argmax(pre_test, axis=1), np.argmax(y_test, axis=1),average='weighted'))

In [ ]:
pre_train= model.predict(x_train)

In [ ]:
print("训练集F1:{:.3f}".format(f1_score(np.argmax(pre_train, axis=1), np.argmax(y_train, axis=1),average='weighted')))
print("测试集F1:{:.3f}".format(f1_score(np.argmax(pre_test, axis=1), np.argmax(y_test, axis=1),average='weighted')))

In [ ]:
test_example = url_change('filedisk.cn')
print(url_change('filedisk.cn'))
print(url_change('http://www.filedisk.cn'))
print(test_example)
x = np.expand_dims(np.array([test_example,test_example]), axis=-1)
model.predict(x)[0]#url为01档的概率
np.argmax(model.predict(x), axis=1)[0]#url为0档

In [ ]:
string = 'http://www.filedisk.cn'
print(string.replace('www.','').replace('http://','').replace('https://',''))
print(string.replace(r'http://',''))